In [3]:
#load modules
from Bio import Entrez, Seq, SeqIO, pairwise2, Align
import numpy as np

#load nucleotide sequences from NCBI 
def get_seq(db, key, rettype="gb"):
    Entrez.email = "A.N.Other@example.com"
    handle = Entrez.efetch(db=db, id=key, rettype=rettype)
    return SeqIO.read(handle, "gb")

## Call NCBI API

In [4]:
#[welche daten werden geladen?]
#hvr-1
nea_1a = get_seq("nucleotide", "AF011222")
nea_1b = get_seq("nucleotide", "AF282971")

#[welche daten werden geladen?]
#hvr-2
nea_2a = get_seq("nucleotide", "AF142095")
nea_2b = get_seq("nucleotide", "AF282972")

In [6]:
n = 206 #known length

nea_1 = nea_1a.seq + nea_1b.seq

## Local Alignments

In [18]:
score = np.zeros(n)
startat = np.zeros(n)

#for i=1:n
#      [sc,alignment,st]=swalign(human_dloop(i).Sequence,nea.Sequence,'Alphabet', 'NT');
aligner = Align.PairwiseAligner()
aligner.mode = 'local'
with open("d_loop.fasta", "r") as handle:
    alignments = []
    d_loop = SeqIO.parse(handle, "fasta")
    for seq in d_loop:
        a = aligner.align(nea_1, seq.seq)
        print(a.score)
        
    
        
        
            
            
"""
from Bio import AlignIO
handle = ...
for a in AlignIO.parse(handle, "fasta-m10"):
  assert len(a) == 2, "Should be pairwise!"
  print("Alignment length %i" % a.get_alignment_length())
  for record in a:
    print("%s %s %s" % (record.seq, record.name, record.id))
-----------------------------------------------------------------
>>> from Bio import Align
>>> aligner = Align.PairwiseAligner()
>>> alignments = aligner.align("ACCGT", "ACG")
>>> for alignment in sorted(alignments):
...     print("Score = %.1f:" % alignment.score)
...     print(alignment)"""

664.0
665.0
665.0
666.0
664.0
668.0
663.0
663.0
662.0
662.0
662.0
662.0
662.0
662.0
664.0
665.0
666.0
665.0
667.0
669.0
670.0
665.0
662.0
666.0
664.0
665.0
661.0
668.0
666.0
666.0
664.0
663.0
667.0
664.0
662.0
667.0
662.0
663.0
661.0
663.0
664.0
663.0
663.0
664.0
667.0
665.0
665.0
662.0
662.0
661.0
665.0
667.0
670.0
665.0
666.0
667.0
666.0
666.0
662.0
665.0
663.0
665.0
665.0
667.0
662.0
662.0
666.0
664.0
666.0
666.0
661.0
663.0
666.0
665.0
665.0
664.0
662.0
665.0
664.0
665.0
666.0
663.0
666.0
666.0
665.0
667.0
664.0
664.0
667.0
667.0
665.0
663.0
665.0
667.0
660.0
665.0
665.0
669.0
664.0
666.0
667.0
664.0
669.0
666.0
664.0
664.0
665.0
665.0
665.0
662.0
666.0
664.0
664.0
667.0
666.0
666.0
667.0
664.0
664.0
664.0
666.0
666.0
665.0
663.0
664.0
666.0
666.0
663.0
663.0
665.0
661.0
667.0
665.0
662.0
668.0
667.0
663.0
667.0
664.0
664.0
668.0
666.0
669.0
661.0
667.0
668.0
661.0
661.0
668.0
665.0
661.0
668.0
667.0
669.0
666.0
664.0
668.0
670.0
662.0
665.0
665.0
666.0
667.0
663.0
670.0
668.0
668.

'\nfrom Bio import AlignIO\nhandle = ...\nfor a in AlignIO.parse(handle, "fasta-m10"):\n  assert len(a) == 2, "Should be pairwise!"\n  print("Alignment length %i" % a.get_alignment_length())\n  for record in a:\n    print("%s %s %s" % (record.seq, record.name, record.id))\n-----------------------------------------------------------------\n>>> from Bio import Align\n>>> aligner = Align.PairwiseAligner()\n>>> alignments = aligner.align("ACCGT", "ACG")\n>>> for alignment in sorted(alignments):\n...     print("Score = %.1f:" % alignment.score)\n...     print(alignment)'

In [27]:
nea_1

Seq('GTTCTTTCATGGGGGAGCAGATTTGGGTACCACCCAAGTATTGACTCACCCATC...CCC', IUPACAmbiguousDNA())

[664.0, 665.0, 665.0, 666.0, 664.0, 668.0, 663.0, 663.0, 662.0, 662.0, 662.0, 662.0, 662.0, 662.0, 664.0, 665.0, 666.0, 665.0, 667.0, 669.0, 670.0, 665.0, 662.0, 666.0, 664.0, 665.0, 661.0, 668.0, 666.0, 666.0, 664.0, 663.0, 667.0, 664.0, 662.0, 667.0, 662.0, 663.0, 661.0, 663.0, 664.0, 663.0, 663.0, 664.0, 667.0, 665.0, 665.0, 662.0, 662.0, 661.0, 665.0, 667.0, 670.0, 665.0, 666.0, 667.0, 666.0, 666.0, 662.0, 665.0, 663.0, 665.0, 665.0, 667.0, 662.0, 662.0, 666.0, 664.0, 666.0, 666.0, 661.0, 663.0, 666.0, 665.0, 665.0, 664.0, 662.0, 665.0, 664.0, 665.0, 666.0, 663.0, 666.0, 666.0, 665.0, 667.0, 664.0, 664.0, 667.0, 667.0, 665.0, 663.0, 665.0, 667.0, 660.0, 665.0, 665.0, 669.0, 664.0, 666.0, 667.0, 664.0, 669.0, 666.0, 664.0, 664.0, 665.0, 665.0, 665.0, 662.0, 666.0, 664.0, 664.0, 667.0, 666.0, 666.0, 667.0, 664.0, 664.0, 664.0, 666.0, 666.0, 665.0, 663.0, 664.0, 666.0, 666.0, 663.0, 663.0, 665.0, 661.0, 667.0, 665.0, 662.0, 668.0, 667.0, 663.0, 667.0, 664.0, 664.0, 668.0, 666.0, 669.0